In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn import preprocessing
import matplotlib.pyplot as plt

## Description of omni2 data:

| No | Type | Missing | Name | Comment |
|----|------------|-------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 1 | I4 |  | Year | 1963, 1964, etc. |
| 2 | I4 |  | Decimal Day | January 1 = Day 1 |
| 3 | I3 |  | Hour | 0, 1,...,23 |
| 4 | I5 | 9999 | Bartels rotation number |  |
| 5 | I3 | 99 | ID for IMF spacecraft | See table |
| 6 | I3 | 99 | ID for SW plasma spacecraft | See table |
| 7 | I4 | 999 | # of points in the IMF averages |  |
| 8 | I4 | 999 | # of points in the plasma averages |  |
| 9 | F6.1 | 999.9 | Field Magnitude Average |B| | 1/N SUM |B|, nT |
| 10 | F6.1 | 999.9 | Magnitude of Average Field Vector $\sqrt{Bx^2+By^2+Bz^2}$ |  |
| 11 | F6.1 | 999.9 | Lat.Angle of Aver. Field Vector | Degrees (GSE coords) |
| 12 | F6.1 | 999.9 | Long.Angle of Aver.Field Vector | Degrees (GSE coords) |
| 13 | F6.1 | 999.9 | Bx GSE, GSM | nT |
| 14 | F6.1 | 999.9 | By GSE | nT |
| 15 | F6.1 | 999.9 | Bz GSE | nT |
| 16 | F6.1 | 999.9 | By GSM | nT |
| 17 | F6 1 | 999.9 | Bz GSM | nT  See http://geo.phys.spbu.ru/~tsyganenko/Geopack-2008.html  developed by Drs. Nikolai Tsyganenko. |
| 18 | F6.1 | 999.9 | sigma|B| | RMS Standard Deviation in average magnitude (word 10), nT |
| 19 | F6.1 | 999.9 | sigma B | RMS Standard Deviation in field vector, nT ($**$) |
| 20 | F6.1 | 999.9 | sigma Bx | RMS Standard Deviation in GSE X-component average, nT |
| 21 | F6.1 | 999.9 | sigma By | RMS Standard Deviation in GSE Y-component average, nT |
| 22 | F6.1 | 999.9 | sigma Bz | RMS Standard Deviation in GSE Z-component average, nT |
| 23 | F9.0 | 9999999. | Proton temperature | Degrees, K |
| 24 | F6.1 | 999.9 | Proton Density | N/cm^3 |
| 25 | F6.0 | 9999. | Plasma (Flow) speed | km/s |
| 26 | F6.1 | 999.9 | Plasma Flow Long. Angle | Degrees, quasi-GSE ($*$) |
| 27 | F6.1 | 999.9 | Plasma Flow Lat. Angle | Degrees, GSE ($*$) |
| 28 | F6.3 | 9.999 | Na/Np | Alpha/Proton ratio |
| 29 | F6.2 | 99.99 | Flow Pressure | P (nPa) = $(1.67/10^6) * Np*V^2 * (1+ 4*Na/Np)$ for hours with non-fill Na/Np ratios and P (nPa) = $(2.0/10^6) * Np*V^2$ for hours with fill values for Na/Np |
| 30 | F9.0 | 9999999. | sigma T | Degrees, K |
| 31 | F6.1 | 999.9 | sigma N | N/cm^3 |
| 32 | F6.0 | 9999. | sigma V | km/s |
| 33 | F6.1 | 999.9 | sigma phi V | Degrees |
| 34 | F6.1 | 999.9 | sigma theta V | Degrees |
| 35 | F6.3 | 9.999 | sigma-Na/Np |  |
| 36 | F7.2 | 999.99 | Electric field | $-[V(km/s) * Bz (nT; GSM)] * 10^{-3}$. (mV/m) |
| 37 | F7.2 | 999.99 | Plasma beta | Beta = $[(T*4.16/10^5) + 5.34] * Np / B^2$ |
| 38 | F6.1 | 999.9 | Alfven mach number | $Ma = (V * Np^{0.5}) / 20 * B$ |
| 39 | I3 | 99 | Kp | Planetary Geomagnetic Activity Index (e.g. 3+ = 33, 6- = 57, 4 = 40, etc.) |
| 40 | I4 | 999 | R | Sunspot number (new version 2) |
| 41 | I6 | 99999 | DST Index | nT, from Kyoto |
| 42 | I5 | 9999 | AE-index | nT, from Kyoto |
| 43 | F10.2 | 999999.99 | Proton flux  | number/cmsq sec sr >1 Mev |
| 44 | F9.2 | 99999.99 | Proton flux | number/cmsq sec sr >2 Mev |
| 45 | F9.2 | 99999.99 | Proton flux | number/cmsq sec sr >4 Mev |
| 46 | F9.2 | 99999.99 | Proton flux | number/cmsq sec sr >10 Mev |
| 47 | F9.2 | 99999.99 | Proton flux | number/cmsq sec sr >30 Mev |
| 48 | F9.2 | 99999.99 | Proton flux | number/cmsq sec sr >60 Mev |
| 49 | I3 | 0 | Flag($***$) | (-1,0,1,2,3,4,5,6) |
| 50 | I4 | 999 | ap-index | nT |
| 51 | F6.1 | 999.9 | f10.7_index | (sfu = 10-22W.m-2.Hz-1) |
| 52 | F6.1 | 999.9 | PC(N) index  |  |
| 53 | I6 | 99999 | AL-index, from Kyoto | nT |
| 54 | I6 | 99999 | AU-index, from Kyoto | nT |
| 55 | F5.1 | 99.9 | Magnetosonic mach number= = V/Magnetosonic_speed Magnetosonic speed = $[(sound speed)^2 + (Alfv speed)^2]^{0.5}$ The Alfven speed = $20. * B / N^{0.5}$ The sound speed = $0.12 * [T + 1.28*10^5]^{0.5}$  About Magnetosonic speed check http://ftpbrowser.gsfc.nasa.gov/bow_derivation1.html also |  |

In [2]:
col_names=["year","decyear","hour","bartels","idemf","idsw","Nimf","Nplasma","B_mean","B_mag","lat","long",
           "Bx_GSE","By_GSE","Bz_GSE","By_GSM","Bz_GSM","sigma","sigma_B","sigma_Bx","sigma_By","sigma_Bz",
           "Proton_Temp","Proton_dens","Plasma_speed","Plasma_flow_lat","Plasma_flow_long","AP_ratio",
           "Flow_pres","sigma_T","sigma_N","sigma_V","sigma_phiV","sigma_thetaV","sigma_ratio","E",
           "Plasma_beta","Alfven_MN","Kp","R","DST","AE_index","Proton_flux1","Proton_flux2","Proton_flux4",
           "Proton_flux10","Proton_flux30","Proton_flux60","Flag","Ap","f10_7","PC_index","AL_index","AU_index",
           "Mag_MN"]
omni=pd.read_csv("data/omni_01_av.dat",delimiter="\s+",names=col_names) #carga la data
omni=omni[(omni["year"]>=1964)&(omni["year"]<=2018)].reset_index(drop=True) #Filtra de 1964 a 2018
# Para tener el tiempo de igual forma que los datos usados para la correlación
ssn = pd.read_csv("data/SN_d_tot_V2.0.csv", delimiter = ";" , header= 0)
time = np.array(ssn.decimal) #Tiempo en formato decimal
omni.head()

,year,decyear,hour,bartels,idemf,idsw,Nimf,Nplasma,B_mean,B_mag,...,Proton_flux10,Proton_flux30,Proton_flux60,Flag,Ap,f10_7,PC_index,AL_index,AU_index,Mag_MN
0,1964,1,0,1785,99,99,6,12,7.8,5.9,...,99999.99,99999.99,99999.99,-1,6,999.9,999.9,99999,99999,99.9
1,1964,2,0,1785,99,99,14,21,8.3,6.4,...,99999.99,99999.99,99999.99,-1,53,68.3,999.9,99999,99999,99.9
2,1964,3,0,1785,99,99,24,24,4.5,3.1,...,99999.99,99999.99,99999.99,-1,21,70.7,999.9,99999,99999,99.9
3,1964,4,0,1785,99,99,13,15,4.6,3.3,...,99999.99,99999.99,99999.99,-1,19,70.4,999.9,99999,99999,99.9
4,1964,5,0,1785,99,99,12,12,3.5,2.1,...,99999.99,99999.99,99999.99,-1,8,71.3,999.9,99999,99999,99.9


In [3]:
#Extraer las variables que se van a utilizar desde el omni
omni_def=omni[["B_mag","Proton_Temp","Proton_dens","Plasma_speed","AP_ratio","Flow_pres","E",
               "Alfven_MN","Kp","R","DST","AE_index","Proton_flux1","Proton_flux2","Proton_flux4",
               "Proton_flux10","Proton_flux30","Proton_flux60","Ap","PC_index","AL_index","AU_index","Mag_MN"]]


In [4]:
#Tratamiendo de los datos
omni_def.loc[omni_def["B_mag"] >= 999.9,"B_mag"] = np.nan
omni_def.loc[omni_def["Proton_Temp"]>=9999999,"Proton_Temp"]=np.nan
omni_def.loc[omni_def["Proton_dens"]>=999.9,"Proton_dens"]=np.nan
omni_def.loc[omni_def["Plasma_speed"]>=9999,"Plasma_speed"]=np.nan
omni_def.loc[omni_def["AP_ratio"]>=9.999,"AP_ratio"]=np.nan
omni_def.loc[omni_def["Flow_pres"]>=99.99,"Flow_pres"]=np.nan
omni_def.loc[omni_def["E"]>=999.99,"E"]=np.nan
omni_def.loc[omni_def["Alfven_MN"]>=999.9,"Alfven_MN"]=np.nan
omni_def.loc[omni_def["Kp"]>=99,"Kp"]=np.nan
omni_def.loc[omni_def["R"]>=999,"R"]=np.nan
omni_def.loc[omni_def["DST"]>=99999,"DST"]=np.nan
omni_def.loc[omni_def["AE_index"]>=9999,"AE_index"]=np.nan
omni_def.loc[omni_def["Proton_flux1"]>=999999.99,"Proton_flux1"]=np.nan
omni_def.loc[omni_def["Proton_flux2"]>=99999.99,"Proton_flux2"]=np.nan
omni_def.loc[omni_def["Proton_flux4"]>=99999.99,"Proton_flux4"]=np.nan
omni_def.loc[omni_def["Proton_flux10"]>=99999.99,"Proton_flux10"]=np.nan
omni_def.loc[omni_def["Proton_flux30"]>=99999.99,"Proton_flux30"]=np.nan
omni_def.loc[omni_def["Proton_flux60"]>=99999.99,"Proton_flux60"]=np.nan
omni_def.loc[omni_def["Ap"]>=999,"Ap"]=np.nan
omni_def.loc[omni_def["PC_index"]>=999.9,"PC_index"]=np.nan
omni_def.loc[omni_def["AL_index"]>=99999,"AL_index"]=np.nan
omni_def.loc[omni_def["AU_index"]>=99999,"AU_index"]=np.nan
omni_def.loc[omni_def["Mag_MN"]>=99.9,"Mag_MN"]=np.nan
#Tratamiento para los que son cero
omni_def["Proton_Temp"].iloc[0]=omni_def["Proton_Temp"].mean()
omni_def["AP_ratio"].iloc[0]=omni_def["AP_ratio"].mean()
omni_def["Proton_flux1"].iloc[0]=omni_def["Proton_flux1"].mean()
omni_def["Proton_flux2"].iloc[0]=omni_def["Proton_flux2"].mean()
omni_def["Proton_flux4"].iloc[0]=omni_def["Proton_flux4"].mean()
omni_def["Proton_flux10"].iloc[0]=omni_def["Proton_flux10"].mean()
omni_def["Proton_flux30"].iloc[0]=omni_def["Proton_flux30"].mean()
omni_def["Proton_flux60"].iloc[0]=omni_def["Proton_flux60"].mean()
omni_def["PC_index"].iloc[0]=omni_def["PC_index"].mean()
omni_def["AL_index"].iloc[0]=omni_def["AL_index"].mean()
omni_def["AU_index"].iloc[0]=omni_def["AU_index"].mean()
omni_def["Mag_MN"].iloc[0]=omni_def["Mag_MN"].mean()


C:\Users\Miguel\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Miguel\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Miguel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas

In [5]:
#Coloca el tiempo como el indice del DataFrame
omni_def.set_index(time, inplace=True)
omni_def.head()

,B_mag,Proton_Temp,Proton_dens,Plasma_speed,AP_ratio,Flow_pres,E,Alfven_MN,Kp,R,...,Proton_flux2,Proton_flux4,Proton_flux10,Proton_flux30,Proton_flux60,Ap,PC_index,AL_index,AU_index,Mag_MN
1964.001,5.9,106557.197131,30.7,328.0,0.039519,6.82,-1.46,11.8,17.0,0.0,...,51.17364,21.266528,7.525882,2.186931,1.06652,6.0,1.069677,-126.64242,78.335786,5.635222
1964.004,6.4,NaN,10.9,505.0,NaN,5.30,0.55,9.1,47.0,21.0,...,NaN,NaN,NaN,NaN,NaN,53.0,NaN,NaN,NaN,NaN
1964.007,3.1,NaN,4.3,499.0,NaN,2.15,0.22,11.9,37.0,12.0,...,NaN,NaN,NaN,NaN,NaN,21.0,NaN,NaN,NaN,NaN
1964.010,3.3,NaN,4.2,492.0,NaN,2.05,0.82,11.3,33.0,11.0,...,NaN,NaN,NaN,NaN,NaN,19.0,NaN,NaN,NaN,NaN
1964.012,2.1,NaN,4.6,375.0,NaN,1.31,0.47,11.8,20.0,19.0,...,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN


In [7]:
# Modo uno
omni_def1=round(omni_def.interpolate().rolling(500,center=True,min_periods=10).mean(),2)
omni_def1.set_index(time, inplace=True)
omni_def1.head()

,B_mag,Proton_Temp,Proton_dens,Plasma_speed,AP_ratio,Flow_pres,E,Alfven_MN,Kp,R,...,Proton_flux2,Proton_flux4,Proton_flux10,Proton_flux30,Proton_flux60,Ap,PC_index,AL_index,AU_index,Mag_MN
1964.001,4.18,93147.94,9.98,404.87,0.04,2.2,-0.04,9.65,19.72,16.10,...,49.25,20.47,7.03,2.08,1.04,10.75,1.05,-107.80,66.70,5.85
1964.004,4.18,93094.09,9.98,405.79,0.04,2.2,-0.04,9.65,19.80,16.08,...,49.24,20.47,7.02,2.08,1.04,10.82,1.05,-107.72,66.65,5.85
1964.007,4.18,93040.23,9.98,406.66,0.04,2.2,-0.04,9.66,19.87,16.13,...,49.23,20.47,7.02,2.08,1.04,10.87,1.05,-107.65,66.60,5.85
1964.010,4.18,92986.38,9.97,407.50,0.04,2.2,-0.04,9.66,19.91,16.14,...,49.22,20.46,7.02,2.08,1.04,10.89,1.05,-107.57,66.56,5.85
1964.012,4.17,92932.53,9.97,408.12,0.04,2.2,-0.04,9.66,19.91,16.14,...,49.22,20.46,7.02,2.08,1.04,10.89,1.05,-107.50,66.51,5.85


In [8]:
#Modo dos
omni_def2=round(omni_def.ewm(com=200, ignore_na=True).mean(),2)
omni_def2.set_index(time, inplace=True)
omni_def2.head()

,B_mag,Proton_Temp,Proton_dens,Plasma_speed,AP_ratio,Flow_pres,E,Alfven_MN,Kp,R,...,Proton_flux2,Proton_flux4,Proton_flux10,Proton_flux30,Proton_flux60,Ap,PC_index,AL_index,AU_index,Mag_MN
1964.001,5.90,106557.2,30.70,328.00,0.04,6.82,-1.46,11.80,17.00,0.00,...,51.17,21.27,7.53,2.19,1.07,6.00,1.07,-126.64,78.34,5.64
1964.004,6.15,106557.2,20.78,416.72,0.04,6.06,-0.45,10.45,32.04,10.53,...,51.17,21.27,7.53,2.19,1.07,29.56,1.07,-126.64,78.34,5.64
1964.007,5.13,106557.2,15.26,444.28,0.04,4.75,-0.23,10.93,33.70,11.02,...,51.17,21.27,7.53,2.19,1.07,26.69,1.07,-126.64,78.34,5.64
1964.010,4.67,106557.2,12.47,456.30,0.04,4.07,0.04,11.03,33.52,11.01,...,51.17,21.27,7.53,2.19,1.07,24.75,1.07,-126.64,78.34,5.64
1964.012,4.15,106557.2,10.88,439.88,0.04,3.51,0.12,11.18,30.79,12.63,...,51.17,21.27,7.53,2.19,1.07,21.37,1.07,-126.64,78.34,5.64


In [31]:
#guardar el archivo como un DataFrame
omni_def1.to_csv('data/Data_for_PCA_mod1.csv')
omni_def2.to_csv('data/Data_for_PCA_mod2.csv')